In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score

In [ ]:
df = pd.read_csv("dataset_eeg.csv")

In [ ]:
df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [ ]:
df = df.set_index(['id_user','session','video'])

In [ ]:
df

## Split train e test
### Subject Dependent
16 trials train e 8 test
2 testing group for labels 

### Subject Indipendent
Leave One Out Cross Validation
#### GradientBoostingClassifier

In [ ]:
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)

In [ ]:
loo = LeaveOneOut()

In [ ]:
scores = cross_val_score(model, df.drop('emotion',axis=1), df.emotion, cv=loo)

### Subject-Biased Experiment
train_test_split

In [ ]:
X_trainSB, X_testSB, y_trainSB, y_testSB = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=42)

In [ ]:
X_testSB

#### GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_trainSB, y_trainSB)

In [ ]:
clf.score(X_testSB, y_testSB)

#### SVM
neutral, sad, fear, and happy => 0 , 1 , 2 , 3

In [ ]:
neutral = svm.SVC()
sad = svm.SVC()
fear = svm.SVC()
happy = svm.SVC()